# Basic Plotting

In [2]:
import xarray as xr
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.path as mpath
from matplotlib.gridspec import GridSpec
from matplotlib.lines import Line2D
from mpl_toolkits.axes_grid1 import make_axes_locatable
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cftime
import os

ModuleNotFoundError: No module named 'xarray'

Use the file: /glade/scratch/hannay/archive/b1850_high_freq/atm/hist/b1850_high_freq.cam.h0.0001-01.nc

In [ ]:
### Here we point to the CESM2-LE datasets on campaign disk

monthly_output_path = "/glade/scratch/hannay/archive/b1850_high_freq/atm/hist"
run_name = "b1850_high_freq.cam.h0.0001-01.nc"

files = os.path.join(monthly_output_path,
                         run_name)
files

In [ ]:
ds = xr.open_dataset(files)
ds

## Exercise 1: Make a lat-lon plot of TS

In [ ]:
# Grab data from first (and only in this case) time stamp
ds['TS'].sel({'time':ds.TS.time.values[0]}).squeeze()

In [ ]:
# Grab a subset of the data for only TS (surface temperatures)
ds.TS

In [ ]:
# How you can grab the variable long name for plot title
ds.TS.long_name

In [ ]:
# How you can grab the times in the dataset
ds.TS.time.values

In [ ]:
# define the colormap
cmap = plt.cm.get_cmap('jet')  

# set up the figure with a Plate Carree projection
fig = plt.figure(figsize=(15,10))

ax = fig.add_subplot(1,1,1, projection=ccrs.PlateCarree())

# sets the latitude / longitude boundaries of the plot
ax.set_extent([0.005, 360, 90, -90], crs=ccrs.PlateCarree())

#Plot the first timeslice of aice
img=ax.pcolormesh(ds['lon'],
                   ds['lat'],
                   ds['TS'].sel({'time':ds.TS.time.values[0]}).squeeze(),
                   cmap=cmap,#vmax=1,vmin=0,
                   transform=ccrs.PlateCarree())

plt.title("Surface Temperature",fontsize=20)

# Set up colorbar
plt.colorbar(img,orientation='vertical',fraction=0.0242,pad=0.01)
plt.show()

In [ ]:
# Xarray native plotting

ts = ds['TS'].sel({'time':ds.TS.time.values[0]}).squeeze()

fig, ax = plt.subplots(1,figsize=(20,10))

ts.plot.contourf(cmap="jet",levels=np.arange(220,321,5))
plt.show()

#### Question:

How is this map oriented? Are there features that stand out? How could we change the central longitude? 

---

## Exercise 2: Zonal plot of SWCF

In [ ]:
ds_swcf = ds.SWCF

d = ds_swcf.dims
davgovr = [dim for dim in d if dim not in ('lev','lat')]

DS = ds_swcf.mean(dim=davgovr)

In [ ]:
fig, ax = plt.subplots(1,figsize=(12,7))

# Set Main title for subplots:
plt.title("Short Wave Cloud Forcing",fontsize=20)

# Set legend for plot line
#TODO: what should go for the line legend, if anything? Maybe time?
line = Line2D([0], [0], label=run_name[:-3],
                        color="#1f77b4")

fig.legend(handles=[line],bbox_to_anchor=(-0.15, 0.15, 1.05, .102),loc="right",
                   borderaxespad=0.0,fontsize=16,frameon=False)

ax.plot(DS['lat'], DS, c="#1f77b4")

ax.set_xlim([max([DS.lat.min(), -90.]), min([DS.lat.max(), 90.])])

ax.set_xlabel("Latitude")
plt.show()

---

## Exercise 3: Plot of zonal T 

In [ ]:
ds_t = ds.T.squeeze()

#### Natively via xarray

 - y-axis is <strong>increasing</strong> with height
 - y-axis is <strong>not</strong> in log pressure

In [ ]:
#Average over all dimensions except `lev` and `lat`.
d = ds_t.dims
davgovr = [dim for dim in d if dim not in ('lev','lat')]

DS = ds_t.mean(dim=davgovr)
DS.transpose('lat', 'lev',...)
fig, ax = plt.subplots(1,figsize=(20,10))
DS.plot.contourf(ax=ax, y='lev',cmap="Spectral_r")
plt.show()

#### ADF style quick plot

 - y-axis is <strong>increasing</strong> with height
 - y-axis is <strong>not</strong> in log pressure

In [ ]:
#Average over all dimensions except `lev` and `lat`.
d = ds_t.dims
davgovr = [dim for dim in d if dim not in ('lev','lat')]

DS = ds_t.mean(dim=davgovr)

lev = DS['lev']
lat = DS['lat']
mlev, mlat = np.meshgrid(lev, lat)

# Generate zonal plot:
fig, ax = plt.subplots(1,figsize=(15,7))

# Create zonal plot with vertical levels 
img = ax.contourf(mlat, mlev, DS.transpose('lat', 'lev'), cmap='Spectral_r')

# Format axis and ticks
ax.set_xlabel("Latitude")
fig.colorbar(img, ax=ax, location='right')

fig.savefig("zonal_vert_T_ADF.png", bbox_inches='tight', dpi=300)

#### Same as above, but:

 - y-axis is <strong>decreasing</strong> with height
 - y-axis is <strong>not</strong> in log pressure

In [ ]:
#Average over all dimensions except `lev` and `lat`.
d = ds_t.dims
davgovr = [dim for dim in d if dim not in ('lev','lat')]

DS = ds_t.mean(dim=davgovr)

#print(DS.lev.min(),DS.lev.max())

lev = DS['lev']
lat = DS['lat']
mlev, mlat = np.meshgrid(lev, lat)

# Generate zonal plot:
fig, ax = plt.subplots(1,figsize=(15,7))

# Create zonal plot with vertical levels 
img = ax.contourf(mlat, mlev, DS.transpose('lat', 'lev'), cmap='Spectral_r')

# Format axis and ticks
plt.gca().invert_yaxis()
ax.tick_params(which='minor', length=4, color='r')

# Set up colorbar
cbar_ax = fig.add_axes([0, 0, 0.1, 0.1])
posn = ax.get_position()

#Set position and size of colorbar position: [left, bottom, width, height]
cbar_ax.set_position([posn.x0 + posn.width + 0.005, posn.y0,
                          0.02, posn.height])

ax.set_xlabel("Latitude")
fig.colorbar(img, cax=cbar_ax,orientation='vertical')


#### Same as above, but:

 - y-axis is <strong>decreasing</strong> with height
 - y-axis <strong>is</strong> in log pressure

In [ ]:
#Average over all dimensions except `lev` and `lat`.
d = ds_t.dims
davgovr = [dim for dim in d if dim not in ('lev','lat')]

DS = ds_t.mean(dim=davgovr)

#print(DS.lev.min(),DS.lev.max())

lev = DS['lev']
lat = DS['lat']
mlev, mlat = np.meshgrid(lev, lat)

# Generate zonal plot:
fig, ax = plt.subplots(1,figsize=(15,7))

# Create zonal plot with vertical levels 
img = ax.contourf(mlat, mlev, DS.transpose('lat', 'lev'), cmap='Spectral_r')

# Format axis and ticks
plt.yscale("log")
plt.gca().invert_yaxis()
ax.tick_params(which='minor', length=4, color='r')

# Set up colorbar
cbar_ax = fig.add_axes([0, 0, 0.1, 0.1])
posn = ax.get_position()

#Set position and size of colorbar position: [left, bottom, width, height]
cbar_ax.set_position([posn.x0 + posn.width + 0.005, posn.y0,
                          0.02, posn.height])

ax.set_xlabel("Latitude")
fig.colorbar(img, cax=cbar_ax,orientation='vertical')